In [1]:
import requests
import os
import pandas as pd
import json

In [2]:
response = requests.get('http://api.citybik.es/v2/networks')
print(response.status_code)

200


In [3]:
def get_city_bike_data(city_name):
    url = 'http://api.citybik.es/v2/networks/'
   
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        networks = data.get('networks',[])
        for network in networks:
            if network['location']['city'].lower() == city_name.lower():
                return network
        print(f"'{city_name}' not found in the CityBikes API.")
        return None
    else:
        print(f"Error: Unable to fetch data for {city_name}. Status code: {response.status_code}")
        return None

In [4]:
city_name = 'Vancouver'
city_bike_data = get_city_bike_data(city_name)

if city_bike_data is not None:
    print(json.dumps(city_bike_data, indent=4))

{
    "company": [
        "Vanncouver Bike Share Inc.",
        "CycleHop LLC",
        "City of Vancouver",
        "Shaw Communications Inc.",
        "Fifteen"
    ],
    "gbfs_href": "https://vancouver-gbfs.smoove.pro/gbfs/2/gbfs.json",
    "href": "/v2/networks/mobibikes",
    "id": "mobibikes",
    "location": {
        "city": "Vancouver",
        "country": "CA",
        "latitude": 49.2827,
        "longitude": -123.1207
    },
    "name": "Mobi"
}


In [5]:
def get_bike_station_details(city_name):
    url = f'http://api.citybik.es/v2/networks/{network_id}'
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        return data.get('network', None)
    else:
        print(f"Error: Unable to fetch data for {network_id}. Status code: {response.status_code}")
        return None
    
network_id = 'mobibikes'
bike_station_details = get_bike_station_details(network_id)
    
if bike_station_details is not None:
    stations = bike_station_details.get('stations', [])
    if not stations:
        print(f"No bike stations data available for {network_id}.")
    else:
        station_details_list = []
        for station in stations:
            station_name = station['name']
            latitude = station['latitude']
            longitude = station['longitude']
            num_bikes = station['free_bikes']
            empty_slots = station['empty_slots']
            station_id = station['id']
            
            station_details_list.append({
                'Station Name': station_name,
                # 'Station ID': station_id,   #didn't want to show it to cluster data...
                'Latitude': latitude,
                'Longitude': longitude,
                'Available Bikes': num_bikes,
                'Empty Slots': empty_slots
            })
        df_city_bike = pd.DataFrame(station_details_list)
else:
    print(f"No data available for {network_id} or network ID not found.")

In [6]:
df_city_bike

,Station Name,Latitude,Longitude,Available Bikes,Empty Slots
0,10th & Cambie,49.262487,-123.114397,14,21
1,Yaletown-Roundhouse Station,49.274566,-123.121817,13,3
2,Dunsmuir & Beatty,49.279764,-123.110154,12,13
3,12th & Yukon (City Hall),49.260599,-123.113504,5,11
4,8th & Ash,49.264215,-123.117772,14,2
...,...,...,...,...,...
240,Cordova & Granville,49.285670,-123.112543,17,0
241,22nd & Main,49.250940,-123.101306,8,6
242,PNE - Hastings & Windermere,49.280977,-123.035969,5,35
243,1st & Fir,49.270783,-123.141564,8,12


### YELP-Foursquare_EDA

In [7]:
api_key = os.environ["location"]
client_id = os.environ["client_id"]
client_secret = os.environ["client_secret"]
api_version = '20230730'

In [8]:
location = "Vancouver, Canada"

In [9]:
url = "https://api.foursquare.com/v3/places/search?near=" + location

In [10]:
# Create dictionary for headers
headers = {"Accept": "application/json"}

# Add key with our API KEY
headers['Authorization'] = api_key

In [11]:
result = requests.get(url, headers=headers)
print(result)

<Response [200]>


In [12]:
data = result.json()
json_data = json.dumps(data, indent=4)
# print(json_data)

# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [17]:
city_name = "Vancouver, Canada"

# '49.2827,-123.1207'
def foursquare_bike_stations(city_name, latitude, longitude, api_key):
    url = f"https://api.foursquare.com/v3/places/search"

    params = {
        'll': f"{latitude},{longitude}",  # Latitude and longitude of the bike station
        'radius': 300,
        'categories': "13033",   # foursquare category for bubble tea shops
        'limit': 5
    }

    headers = {"Accept": "application/json"}
    headers['Authorization'] = api_key

    response = requests.get(url, params=params, headers=headers)

    if response.status_code == 200:
        data = response.json()
        bike_stations = data.get('results', [])

        return bike_stations
    else:
        print(f"Error: Unable to fetch Foursquare data. Status code: {response.status_code}")
        return []

# Function to collect the details of the bubble tea shops for each bike station
def bubble_tea_details(station_name, bubble_tea_shops):
    details_list = []
    for idx, shop in enumerate(bubble_tea_shops, start=1):
        name = shop.get('name', 'N/A')
        rating = shop.get('rating', 'N/A')
        location = shop.get('location', {})
        shop_latitude = location.get('lat', 'N/A')
        shop_longitude = location.get('lng', 'N/A')

        details_list.append({
            'Station Name': station_name,
            'Bubble Tea Shop Name': name,
            'Rating': rating,
            'Latitude': shop_latitude,
            'Longitude': shop_longitude
        })

    return details_list

# Create an empty list to store the details of bubble tea shops for all bike stations
all_bubble_tea_details = []

# Loop through each bike station and get bubble tea shop details
for index, row in df_city_bike.iterrows():
    station_name = row['Station Name']
    latitude = row['Latitude']
    longitude = row['Longitude']

    bubble_tea_shops = foursquare_bike_stations(city_name, latitude, longitude, api_key)

    # Collect bubble tea shop details for the current bike station
    station_bubble_tea_details = bubble_tea_details(station_name, bubble_tea_shops)

    # Add the details to the list
    all_bubble_tea_details.extend(station_bubble_tea_details)

# Create a DataFrame from the collected bubble tea shop details
df_bubble_tea = pd.DataFrame(all_bubble_tea_details)

# Print DataFrame
df_bubble_tea

,Station Name,Bubble Tea Shop Name,Rating,Latitude,Longitude
0,10th & Cambie,OneZo Tapioca,N/A,N/A,N/A
1,Yaletown-Roundhouse Station,ShareTea Yaletown,N/A,N/A,N/A
2,Yaletown-Roundhouse Station,Cha Yuan,N/A,N/A,N/A
3,Dunsmuir & Beatty,Bubble King Tea House,N/A,N/A,N/A
4,8th & Ash,OneZo Tapioca,N/A,N/A,N/A
...,...,...,...,...,...
160,Cordova & Granville,Coco Fresh Tea & Juice,N/A,N/A,N/A
161,Cordova & Granville,Teaever Bubble Tea,N/A,N/A,N/A
162,Cordova & Granville,Gong Cha West Pender,N/A,N/A,N/A
163,Cordova & Granville,Chatime,N/A,N/A,N/A


Put your parsed results into a DataFrame

# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [27]:
import requests
import os

yelp_api_key = "pMleMhYHNwkjKxj725WRoV50sEPxvah29T4bQ9spFVCodth9zsyHvxueifmowF6xz-BDCFMxb4ba8THLm83BlhuEdHindRV9VnaZot4c-JgkBCnsuqxaMkDsn-nHZHYx"

url = "https://api.yelp.com/v3/categories"

headers = {
    "accept": "application/json",
    "Authorization": f"Bearer {yelp_api_key}"
}

response = requests.get(url, headers=headers)

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

Put your parsed results into a DataFrame

In [28]:
radius = 300  # 300 meters radius for Yelp API request - reduced to return less results, run faster code

city_name = "Vancouver, Canada"

# Function to send a request to Yelp API
def yelp_bike_stations(city_name, latitude, longitude, yelp_api_key):
    url = f"https://api.yelp.com/v3/businesses/search"

    params = {
        'latitude': latitude,
        'longitude': longitude,
        'radius': radius,
        'categories': "bubbletea",
        'limit': 5
    }

    headers = {"Authorization": f"Bearer {yelp_api_key}"}

    response = requests.get(url, params=params, headers=headers)

    if response.status_code == 200:
        data = response.json()
        bike_stations = data.get('businesses', [])

        return bike_stations
    else:
        print(f"Error: Unable to fetch Yelp data. Status code: {response.status_code}")
        return []

# collect details of Yelp businesses for each bike station
def collect_yelp_details(station_name, bike_stations):
    details_list = []
    for idx, station in enumerate(bike_stations, start=1):
        station_latitude = station.get('coordinates', {}).get('latitude', 'N/A')
        station_longitude = station.get('coordinates', {}).get('longitude', 'N/A')

        # Fetch Yelp business details for the current bike station
        yelp_businesses = yelp_bike_stations(city_name, station_latitude, station_longitude, yelp_api_key)

        if yelp_businesses:
            for idx, business in enumerate(yelp_businesses, start=1):
                name = business.get('name', 'N/A')
                rating = business.get('rating', 'N/A')
                business_latitude = business.get('coordinates', {}).get('latitude', 'N/A')
                business_longitude = business.get('coordinates', {}).get('longitude', 'N/A')

                details_list.append({
                    'Station Name': station_name,
                    'Station Latitude': station_latitude,
                    'Station Longitude': station_longitude,
                    'Yelp Business Name': name,
                    'Rating': rating,
                    'Business Latitude': business_latitude,
                    'Business Longitude': business_longitude
                })

    return details_list

# Create an empty list to store the details of Yelp businesses for all bike stations
all_yelp_details = []

# Loop through each bike station and get Yelp business details
for index, row in df_city_bike.iterrows():
    station_name = row['Station Name']
    latitude = row['Latitude']
    longitude = row['Longitude']

    bike_stations = yelp_bike_stations(city_name, latitude, longitude, yelp_api_key)

    # Collect Yelp business details for the current bike station
    station_yelp_details = collect_yelp_details(station_name, bike_stations)

    # Add the details to the list
    all_yelp_details.extend(station_yelp_details)

# Create a DataFrame from the collected Yelp business details
df_yelp_businesses = pd.DataFrame(all_yelp_details)

# Print DF
df_yelp_businesses

Error: Unable to fetch Yelp data. Status code: 429
Error: Unable to fetch Yelp data. Status code: 429
Error: Unable to fetch Yelp data. Status code: 429
Error: Unable to fetch Yelp data. Status code: 429
Error: Unable to fetch Yelp data. Status code: 429
Error: Unable to fetch Yelp data. Status code: 429
Error: Unable to fetch Yelp data. Status code: 429
Error: Unable to fetch Yelp data. Status code: 429
Error: Unable to fetch Yelp data. Status code: 429
Error: Unable to fetch Yelp data. Status code: 429
Error: Unable to fetch Yelp data. Status code: 429


KeyboardInterrupt: 

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

##### The YELP API was incredibly more complete. Setting up the code to be similar in both Foursquare and YELP, I found that the YELP one listed all the names, rating, both latitude and longitude, whereas the Foursquare only listed the bubble tea stores by name and returned N/A for rating and both latitude and longitude. 

Get the top 10 restaurants according to their rating (bubbletea shops)

In [25]:
# Sort the DataFrame by 'Yelp Rating' in descending order
sorted_df = df_yelp_businesses.sort_values(by='Yelp Rating', ascending=False)

# Get the top 10 bubbletea shops according to their rating
top_10_bbt_shops = sorted_df.nlargest(10, 'Yelp Rating')

# Print the top 10 restaurants
print(top_10_bbt_shops)

NameError: name 'df_yelp_businesses' is not defined